In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import random
import time
from selenium.webdriver import Firefox
from scrape_functions import (
    get_movie_scores,
    get_all_titles
)

In [3]:
browser = Firefox()
url = "https://www.rottentomatoes.com/browse/in-theaters?minTomato=0&maxTomato=100&minPopcorn=0&maxPopcorn=100&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=popularity"
browser.get(url)

In [4]:
# def get_movie_scores(browser):
#     """Return all movie scores on the page"""
#     sel = "div.mb-movies div.mb-movie div.movie_info a div"
#     scores = browser.find_elements_by_css_selector(sel)
#     return scores
movie_scores_list = get_movie_scores(browser)

In [5]:
movie_scores_list[0].text

' 32% 50%'

In [6]:
scores=[]
for movie in movie_scores_list:
    one_score = movie.text
    scores.append(one_score)

In [7]:
scores

[' 32% 50%',
 ' 90% 87%',
 ' 64% 78%',
 ' 78% 56%',
 ' 46% 74%',
 ' 47% 55%',
 ' 58% 40%',
 ' 94% 66%',
 ' 89% 70%',
 ' 14% 60%',
 ' 20% 78%',
 ' 91% 75%',
 ' 38% 37%',
 ' 91% 87%',
 ' 99% 90%',
 ' 52% 76%',
 ' 48% 92%',
 ' 55% 79%',
 ' 94% 75%',
 ' 75% 84%',
 ' 33% 38%',
 ' 82% 49%',
 ' 71% 81%',
 ' 13% 25%',
 ' 86% 71%',
 ' 98% 90%',
 ' 69% 50%',
 ' 14% 23%',
 ' 39% 93%',
 ' 41% 84%',
 ' 60% 94%',
 ' 90% 75%']

In [8]:
scores_lean = [score.replace(' ','') for score in scores]
scores_lean

['32%50%',
 '90%87%',
 '64%78%',
 '78%56%',
 '46%74%',
 '47%55%',
 '58%40%',
 '94%66%',
 '89%70%',
 '14%60%',
 '20%78%',
 '91%75%',
 '38%37%',
 '91%87%',
 '99%90%',
 '52%76%',
 '48%92%',
 '55%79%',
 '94%75%',
 '75%84%',
 '33%38%',
 '82%49%',
 '71%81%',
 '13%25%',
 '86%71%',
 '98%90%',
 '69%50%',
 '14%23%',
 '39%93%',
 '41%84%',
 '60%94%',
 '90%75%']

In [ ]:
scores_split = [score_lean.split('%') for score_lean in scores_lean]

split_scores = [score_split.pop() for score_split in scores_split]

In [11]:
scores_split

[['32', '50'],
 ['90', '87'],
 ['64', '78'],
 ['78', '56'],
 ['46', '74'],
 ['47', '55'],
 ['58', '40'],
 ['94', '66'],
 ['89', '70'],
 ['14', '60'],
 ['20', '78'],
 ['91', '75'],
 ['38', '37'],
 ['91', '87'],
 ['99', '90'],
 ['52', '76'],
 ['48', '92'],
 ['55', '79'],
 ['94', '75'],
 ['75', '84'],
 ['33', '38'],
 ['82', '49'],
 ['71', '81'],
 ['13', '25'],
 ['86', '71'],
 ['98', '90'],
 ['69', '50'],
 ['14', '23'],
 ['39', '93'],
 ['41', '84'],
 ['60', '94'],
 ['90', '75']]

In [12]:
critics_score = [score[0] for score in scores_split]
audience_score = [score[1] for score in scores_split]

In [13]:
all_titles = get_all_titles(browser)

In [14]:
all_titles

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="315b42f1-dd06-f44c-9fa5-828658f5bbc5", element="afcceabf-ea5e-1f43-adbc-da4ec8fdb741")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="315b42f1-dd06-f44c-9fa5-828658f5bbc5", element="077286b5-35a5-ca49-8785-a233209fb65c")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="315b42f1-dd06-f44c-9fa5-828658f5bbc5", element="b61ec733-9458-1a49-bd9c-3029b337398c")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="315b42f1-dd06-f44c-9fa5-828658f5bbc5", element="635e9150-c079-aa48-a86e-fcaa5effff25")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="315b42f1-dd06-f44c-9fa5-828658f5bbc5", element="bf56dc98-edff-ef44-a51f-d67c8de5ce32")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="315b42f1-dd06-f44c-9fa5-828658f5bbc5", element="d0ba12f7-a17c-934e-8b2e-3e73774d866d")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement 

In [15]:
titles=[]
for movie in all_titles:
    one_title = movie.text
    titles.append(one_title)

In [16]:
titles

['The Curse Of La Llorona',
 'Shazam!',
 'Breakthrough',
 'Captain Marvel',
 'Little',
 'Dumbo',
 'Pet Sematary',
 'Us',
 'Missing Link',
 'Hellboy',
 'After',
 'Disneynature: Penguins',
 'Kalank',
 'How To Train Your Dragon: The Hidden World',
 'Amazing Grace',
 'The Best Of Enemies',
 'Unplanned',
 'Five Feet Apart',
 'The Mustang',
 'Hotel Mumbai',
 'Wonder Park',
 'High Life',
 'Teen Spirit',
 "Tyler Perry's A Madea Family Funeral",
 'The LEGO Movie 2: The Second Part',
 'Apollo 11',
 "Isn't It Romantic",
 'No Manches Frida 2',
 'The Chaperone',
 'The Upside',
 'Alita: Battle Angel',
 'Wild Nights With Emily']

In [17]:
import pandas as pd

In [18]:
dictionary = {'title':titles, 'critics_score':critics_score, 'audience_score':audience_score}

In [19]:
rt_df = pd.DataFrame(dictionary)
rt_df

,title,critics_score,audience_score
0,The Curse Of La Llorona,32,50
1,Shazam!,90,87
2,Breakthrough,64,78
3,Captain Marvel,78,56
4,Little,46,74
5,Dumbo,47,55
6,Pet Sematary,58,40
7,Us,94,66
8,Missing Link,89,70
9,Hellboy,14,60


In [20]:
import pymongo

In [21]:
mc = pymongo.MongoClient()

In [22]:
db = mc['movie']

In [23]:
rt_coll = db['rotten_tomatoes']

In [24]:
rt_coll.insert_one(dictionary)